In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("test_lib_from_postgres_to_clickhouse")
    .config(
        "spark.jars.packages",
        ",".join([
            # clickhouse drivers
            "com.clickhouse:clickhouse-jdbc:0.7.2",
            # postgres drivers
            "org.postgresql:postgresql:42.7.3"
        ])
    )
    .getOrCreate()
)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
com.clickhouse#clickhouse-jdbc added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-dcbb816f-0037-41dd-833d-e79f1cba957e;1.0
	confs: [default]
	found com.clickhouse#clickhouse-jdbc;0.7.2 in central
	found com.clickhouse#clickhouse-client;0.7.2 in central
	found com.clickhouse#clickhouse-data;0.7.2 in central
	found com.clickhouse#clickhouse-http-client;0.7.2 in central
	found org.apache.httpcomponents.core5#httpcore5-h2;5.2 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.httpcomponents.client5#httpclient5;5.2.1 in central
	found org.apache.httpcomponents.core5#httpcore5;5.2.1 in central
	found org.apache.commons#commons-compress;1.27.1 in central
	found commons-codec#commons-codec;1.17.1 in central
	found commons-io#commons-io;2.16.1 in central
	found org.apache.commons

In [2]:
from data_transfer_lib import Postgres, ClickHouse, Reader, Writer

In [5]:
postgres_conn = Postgres(
    host="postgres",
    user="postgres_user",
    password="postgres_password",
    database="postgres",
    port=5432,
    spark=spark
)

reader = Reader(
    connection=postgres_conn,
    db_name="public",
    table_name="test_data"
)

df = reader.start()

In [6]:
df.show()

+---+-------+-----+
| id|   name|value|
+---+-------+-----+
|  1|  Alice|   10|
|  2|    Bob|   20|
|  3|Charlie|   30|
|  4|  Alice|   10|
|  5|    Bob|   20|
|  6|Charlie|   30|
+---+-------+-----+



In [7]:
clickhouse_conn = ClickHouse(
    host="clickhouse-server",
    user="default",
    password="1234qwe",
    database="default",
    spark=spark
)

In [8]:
writer = Writer(
    connection=clickhouse_conn,
    db_name="default",
    table_name="test_table",
    if_exists=True
)

{'id': 'Int32', 'name': 'Nullable(String)', 'value': 'Nullable(Int32)'}


In [9]:
writer.start(
    df=df,
    mode="append",
    batch_size=10000,
    num_partitions=5
)

Validation is succed


25/11/23 16:10:34 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/11/23 16:10:34 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/11/23 16:10:34 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/11/23 16:10:34 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/11/23 16:10:34 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/11/23 16:10:34 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQ

In [10]:
spark.stop()